In [4]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import *
from pyspark.sql.functions import explode, collect_list, size, col, row_number, sort_array, udf, count

In [5]:
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

In [6]:
graphPath = "/data/graphDFSample"

In [7]:
reversedGraph = sparkSession.read.parquet(graphPath) \
    .withColumn("friend", explode('friends')) \
    .groupBy("friend") \
    .agg(collect_list("user").alias("users")) \
    .withColumn("users_size", size("users"))

reversedGraph.show(3)

+------+--------------------+----------+
|friend|               users|users_size|
+------+--------------------+----------+
|   148|[65051219, 146311...|         4|
|  5518|          [58573511]|         1|
|  9900|          [36844066]|         1|
+------+--------------------+----------+
only showing top 3 rows



In [8]:
reversedGraph = reversedGraph.select(reversedGraph.friend, \
                                     sort_array(reversedGraph.users).alias("users_sorted"), \
                                     "users_size") \
                            .where(reversedGraph.users_size > '1')
reversedGraph.show(3)

+------+--------------------+----------+
|friend|        users_sorted|users_size|
+------+--------------------+----------+
|   148|[3195315, 1463110...|         4|
| 36538|[5506394, 6170161...|        32|
| 41751|[41811068, 60873111]|         2|
+------+--------------------+----------+
only showing top 3 rows



In [9]:
reversedGraph.printSchema()

root
 |-- friend: integer (nullable = true)
 |-- users_sorted: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- users_size: integer (nullable = false)



In [10]:
def serializer(input_array):
    out_array = []
    count = 0
    while count < len(input_array):
        for count_internal in range (count+1, len(input_array)):
            out_array.append((input_array[count], input_array[count_internal]))
        count += 1
    return(out_array)

In [11]:
serializer_udf = udf(lambda y: serializer(y), 
                     ArrayType(StructType(
                         (StructField("1", IntegerType(), True),
                          StructField("2", IntegerType(), True)))))

In [12]:
reversedGraph = reversedGraph.select(serializer_udf(reversedGraph.users_sorted).alias("users"), \
                                    "users_size")
reversedGraph.show(3)

+--------------------+----------+
|               users|users_size|
+--------------------+----------+
|[[3195315,1463110...|         4|
|[[5506394,6170161...|        32|
|[[41811068,608731...|         2|
+--------------------+----------+
only showing top 3 rows



In [13]:
reversedGraph.printSchema()

root
 |-- users: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- 1: integer (nullable = true)
 |    |    |-- 2: integer (nullable = true)
 |-- users_size: integer (nullable = false)



In [14]:
reversedGraph_2 = reversedGraph.select(explode(reversedGraph.users), \
                                      "users_size")
reversedGraph_2.show(3)

+------------------+----------+
|               col|users_size|
+------------------+----------+
|[3195315,14631101]|         4|
|[3195315,14957568]|         4|
|[3195315,65051219]|         4|
+------------------+----------+
only showing top 3 rows



In [15]:
reversedGraph_2.printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- 1: integer (nullable = true)
 |    |-- 2: integer (nullable = true)
 |-- users_size: integer (nullable = false)



In [16]:
reversedGraph_3 = reversedGraph_2.select('col.*')
reversedGraph_3.show(10)

+--------+--------+
|       1|       2|
+--------+--------+
| 3195315|14631101|
| 3195315|14957568|
| 3195315|65051219|
|14631101|14957568|
|14631101|65051219|
|14957568|65051219|
| 5506394| 6170161|
| 5506394| 6846874|
| 5506394|15305594|
| 5506394|17133226|
+--------+--------+
only showing top 10 rows



In [17]:
reversedGraph_4 = reversedGraph_3.groupBy("1", "2").agg(count("*").alias("count"))

reversedGraph_4.show(5)

In [20]:
reversedGraph_4.printSchema()

root
 |-- 1: integer (nullable = true)
 |-- 2: integer (nullable = true)
 |-- count: long (nullable = false)



In [23]:
window = Window.orderBy(col("count").desc())
    
top50 = reversedGraph_4.withColumn("row_number", row_number().over(window)) \
            .filter(col("row_number") < 50) \
            .select(col("count"), col("1"), col("2")) \
            .orderBy(col("count").desc(), col("1").desc(), col("2").desc()) \
            .collect()

In [24]:
for val in top50:
    print '%s %s %s' % val

3206 27967558 42973992
3130 20158643 42973992
3066 22582764 42973992
3044 21864412 51640390
3021 17139850 51640390
3010 14985079 51640390
2970 17139850 21864412
2913 20158643 27967558
2903 22280814 51151280
2870 23848749 51640390
2855 20158643 22582764
2849 20158643 44996025
2846 22280814 42973992
2784 21864412 23848749
2779 31964081 51640390
2776 39205988 51640390
2754 17139850 23848749
2749 22582764 27967558
2728 50561859 51640390
2724 15485897 51640390
2700 28135661 42973992
2655 22280814 27967558
2653 42973992 43548989
2639 26755857 51640390
2621 14635589 51640390
2608 15485897 17139850
2606 17139850 26755857
2601 21864412 39205988
2600 8406745 51640390
2599 37735419 51640390
2597 20158643 28135661
2585 40003405 42973992
2585 21864412 31964081
2581 27967558 43548989
2579 23848749 31964081
2578 27967558 28135661
2578 15485897 21864412
2577 42973992 64755069
2574 51151280 57077210
2573 20158643 43548989
2566 21864412 26755857
2564 22280814 64755069
2561 42973992 44996025
2556 1713985